In [1]:
import json
import pandas as pd
import csv
import re
import string
import os
import time
from twitter_credentials import *
from datetime import datetime

# Libraries for Twitter API
import tweepy
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

In [2]:
# Accessing the credentials from twitterapi.py
accesstoken = access_token
accesstokensecret = access_secret_token
apikey = api_key
apisecretkey = api_security_key

# Connecting to Twitter API
auth = OAuthHandler(apikey, apisecretkey)
auth.set_access_token(accesstoken, accesstokensecret)
api = tweepy.API(auth)

In [ ]:
def tweetcollector(search_words, date_since, numTweets, numRuns):
    
    # Define a pandas dataframe to store the data
    df_tweets = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags']
                                )

    for i in range(0, numRuns):
        # We will time how long it takes to scrape tweets for each run:
        
        tweets = tweepy.Cursor(api.search, q=search_words + " -filter:retweets", lang="en", since=date_since, tweet_mode='extended').items(numTweets)

        tweet_list = [tweet for tweet in tweets]

        noTweets = 0
        for tweet in tweet_list:

                    username = tweet.user.screen_name
                    acctdesc = tweet.user.description
                    location = tweet.user.location
                    following = tweet.user.friends_count
                    followers = tweet.user.followers_count
                    totaltweets = tweet.user.statuses_count
                    usercreatedts = tweet.user.created_at
                    tweetcreatedts = tweet.created_at
                    retweetcount = tweet.retweet_count
                    hashtags = tweet.entities['hashtags']
                    
                    try:
                        # Check wether the tweet was re-tweeted.
                        
                        text = tweet.retweeted_status.full_text
                        
                    except AttributeError:
                        
                        # if it Not a Retweeted tweet run the following code 
                        
                        text = tweet.full_text

                        the_tweet = [username, acctdesc, location, following, followers, totaltweets,
                                     usercreatedts, tweetcreatedts, retweetcount, text, hashtags]

                        df_tweets.loc[len(df_tweets)] = the_tweet

                        noTweets += 1
                        
                        #15 minute sleep time because of twitter requests limitation.
                        time.sleep(920) 
                        
                        tot_csv_timesamp = datetime.today().strftime('%Y%m%d_%H%M%S')
                        
                        # Defining a path for storing the collected tweet
                        path = os.getcwd()
                        filename = path + '/COVIDRestriction/' + tot_csv_timesamp + 'covidrestriction.csv'
                        
                        # The pandas dataframe is converted into CSV fil Format.
                        df_tweets.to_csv(filename, index = False)

In [ ]:
# Initialise the variable for the function created

search_words = "COVID19 Restrictions OR Masks OR Social Distancing OR Lockdown"
date_since = "2019-11-03"
NumberOfTweets = 100
NumberOfRuns = 6

# Call the function tweetcollector
tweetcollector(search_words, date_since, NumberOfTweets, NumberOfRuns)